In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from bokeh.plotting import figure, show, output_notebook
import ipywidgets as widgets
from IPython.display import clear_output

API_BASE = 'https://geo.weather.gc.ca/geomet'

# Round a number to the nearest multiple 
def round_multiple(num, multiple):
    return ((num + multiple // 2) // multiple) * multiple

# Retrieving the API datetime, API is available at three hour intervals following midnight, 3am, 6am, 9am etc...
def get_api_datetime(time: datetime) -> str:
    utc_time = datetime.utcfromtimestamp(time.timestamp())
    rounded_hour = round_multiple(utc_time.hour, 3)
    if rounded_hour >= 24:
        utc_time = utc_time.replace(hour=0, minute=0, second=0) + timedelta(days=1)
    else:
        utc_time = utc_time.replace(hour=rounded_hour, minute=0, second=0)
    return utc_time.isoformat(timespec='seconds')

# Retrieving the API URL
def get_api_url(time: datetime, lat: float, lng: float, layers='GDPS.ETA_TT'):
    pred_time = get_api_datetime(time)
    url = (
        'https://geo.weather.gc.ca/geomet?SERVICE=WMS&VERSION=1.3.0'
        '&REQUEST=GetFeatureInfo'
        '&BBOX={lat},{lng},{lat_end},{lng_end}'
        '&CRS=EPSG:4326&WIDTH=10&HEIGHT=10'
        '&LAYERS={layers}'
        '&QUERY_LAYERS={layers}'
        '&INFO_FORMAT=application/json'
        '&I=5&J=5'
        '&TIME={time}Z'
    )
    return url.format(lat=lat, lng=lng, lat_end=lat+0.5, lng_end=lng+0.5, time=pred_time, layers=layers), pred_time

# Function to get the air temperature
def get_air_temperature(time: datetime):
    url, pred_time = get_api_url(time, lat=53, lng=-113) # City of Edmonton
    res = requests.get(url)
    return res.json(), pred_time

def get_air_temp_predictions(num=5) -> pd.DataFrame:
    air_temps = []
    pred_times = []
    now = datetime.now()
    times = [now + timedelta(hours=3 * i) for i in range(num)]
    for time in times:
        air_temp_json, pred_time_str = get_air_temperature(time)
        air_temp = float(air_temp_json['features'][0]['properties']['value'])
        air_temps.append(air_temp)
        pred_times.append(pred_time_str)
    df = pd.DataFrame({'air_temp': air_temps}, index=pd.to_datetime(pred_times))
    return df

#  Plotting the weather data
def plot_weather_data(df):
    output_notebook()
    p = figure(title='Air temperature forecast', x_axis_type='datetime', x_axis_label='Time of day', y_axis_label='Temperature (Celsius)')
    p.line(x=df.index, y=df['air_temp'])
    p.scatter(x=df.index, y=df['air_temp'])
    return p

# Fetching and plot the data
def fetch_and_plot():
    air_temps_df = get_air_temp_predictions() 
    p = plot_weather_data(air_temps_df) 
    return p

# Define a function to be called when the button is pressed
def update(button):
    new_p = fetch_and_plot()
    clear_output(wait=True)
    output_notebook()
    show(new_p, notebook_handle=True)

# Button widget
button = widgets.Button(description="Refresh Data")

# Set the button's callback to the update function
button.on_click(update)

# Fetch and plot the initial data
initial_p = fetch_and_plot()

# Display the initial plot and the button
output_notebook()
show(initial_p, notebook_handle=True)
display(button)


Loading BokehJS ...

Loading BokehJS ...

Button(description='Refresh Data', style=ButtonStyle())